In [11]:
import os
import pandas as pd
import mne
import numpy as np
from pytorchNN import *
from feature_extraction import *
from sklearn.model_selection import train_test_split

In [12]:
# Specify the directory paths
records_csv_path = "EEG data/Zhang & Wamsley 2019/Records.csv"
psg_directory = "EEG data/Zhang & Wamsley 2019/Data/PSG/"

# Read the metadata from the CSV file
rec_df = pd.read_csv(records_csv_path, index_col=0)

# Initialize an empty list to store the mainList arrays for each file
all_main_lists = []

In [13]:

# Loop through all files in the PSG directory
for filename in os.listdir(psg_directory):
    file_path = os.path.join(psg_directory, filename)

    # Read the raw EEG data using MNE
    mne.set_log_level("WARNING")
    data = mne.io.read_raw_edf(file_path)
    dream_df = data.get_data()

    # Extract EEG data for a specific channel (e.g., "Fp1-REF")
    channels = data.ch_names
    raw_df = pd.DataFrame(dream_df)
    raw_df.index = channels
    eeg_df = raw_df.loc["Fp1-REF"].copy().values

    # Split the EEG data into epochs
    list_of_epochs = [eeg_df[i:i + 800] for i in range(0, len(eeg_df), 800)][:9]  # Adjust the number of epochs as needed

    # Initialize an empty list to store the mainList arrays for each file
    main_list_for_file = []

    # Loop through each epoch
    for epoch in list_of_epochs:
        ft_transform = FourierTransform(400, 0.5)  # Adjust parameters as needed
        ft_output, freq_bin = ft_transform.transform(epoch, 2)
        y = np.square(ft_output)
        new = y[:50]
        main_list_for_epoch = np.append(new, 73)  # Append the additional value (73) to the end (duration)
        main_list_for_file.extend(main_list_for_epoch)

    # Append the mainList array for the file to the overall list
    all_main_lists.append(main_list_for_file)

In [14]:
# Convert the list of lists to a Pandas DataFrame
df = pd.DataFrame(all_main_lists)

# Print or use the DataFrame as needed
df

,0,1,2,3,4,5,6,7,8,9,...,449,450,451,452,453,454,455,456,457,458
0,1.171236e-07,7.788929e-06,1.562104e-06,1.279969e-06,3.426676e-06,3.949851e-06,1.149552e-06,2.102898e-06,4.863842e-08,6.113909e-07,...,7.654028e-08,1.162019e-07,5.490025e-08,3.134641e-08,1.037846e-08,3.635815e-08,1.122343e-07,1.657590e-07,7.382882e-10,73.0
1,1.992009e-06,3.313768e-06,2.741260e-06,8.910712e-07,5.881838e-07,1.170998e-06,4.412049e-07,1.022097e-06,2.699555e-07,2.202543e-07,...,1.710161e-08,9.805498e-08,2.942589e-09,1.175239e-08,5.994546e-08,3.189657e-08,4.016301e-08,7.843397e-08,3.674311e-09,73.0
2,6.141061e-08,6.143634e-07,4.494110e-06,9.525036e-06,9.311612e-07,1.417550e-06,5.877069e-06,2.214444e-06,5.818512e-07,1.008286e-06,...,5.019729e-08,8.553185e-08,2.927852e-08,3.280512e-08,1.137960e-09,1.685638e-10,4.630326e-08,6.956952e-10,1.623467e-08,73.0
3,2.032469e-07,4.536365e-06,4.727014e-06,1.885462e-06,1.013685e-06,5.758731e-07,1.890050e-07,3.841409e-07,4.594360e-07,1.283838e-06,...,3.045105e-07,5.951353e-08,3.684909e-08,8.165695e-08,2.681361e-08,2.461162e-07,1.111064e-07,5.065298e-08,7.661032e-08,73.0
4,2.552197e-06,1.492030e-05,9.745138e-07,1.178725e-06,2.853897e-06,1.126445e-06,2.014350e-07,6.636696e-07,1.076188e-07,2.087899e-06,...,3.360252e-08,1.297059e-07,1.144412e-07,8.757719e-08,1.883532e-07,3.993183e-08,1.739286e-08,2.349801e-08,5.261846e-08,73.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,9.210851e-09,1.410293e-07,1.075864e-07,2.995883e-08,3.748722e-08,1.179543e-07,3.800462e-08,2.589703e-08,2.770355e-09,3.526040e-08,...,1.112025e-08,1.656122e-09,4.387391e-09,5.141508e-10,1.117114e-08,1.846138e-09,6.679525e-09,2.571129e-09,2.786253e-09,73.0
304,6.324013e-08,1.464000e-08,2.831734e-07,4.026896e-07,2.552229e-09,1.511347e-07,1.260878e-07,3.405908e-08,8.987289e-08,1.836051e-09,...,1.387010e-09,1.816063e-08,3.845973e-09,8.095770e-09,4.563785e-09,7.360561e-10,4.514232e-09,3.852973e-09,5.122953e-10,73.0
305,3.503967e-10,4.389255e-08,1.025622e-07,1.523971e-07,3.902872e-08,2.675207e-07,1.462619e-07,1.296681e-07,2.679628e-08,1.100116e-08,...,5.481560e-11,1.683533e-09,2.291366e-09,2.256334e-09,5.330103e-09,7.108810e-09,1.348800e-09,2.537827e-09,3.648122e-11,73.0
306,1.302977e-08,3.709132e-07,1.925461e-07,7.340268e-09,2.440892e-08,4.680389e-08,3.281879e-09,4.135481e-09,3.372549e-09,4.146603e-08,...,3.014788e-10,2.475090e-09,3.518762e-10,8.554777e-10,1.789020e-09,1.688478e-09,1.481933e-10,7.402468e-10,3.436675e-09,73.0


In [15]:
# Preparing which columns will be used in input and which ones will be used in output of NN
X = df.iloc[:, :-1]
y = df.iloc[:, -1:]

In [16]:
# Convert df to numpy arrays
X = X.values
y = y.values

In [17]:
# Split Training and Test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [18]:
# Initialize the object of the model
model = Model()

In [19]:
# Selects criterion for which to evaluate the model, I picked L1loss
criterion = nn.L1Loss()
# How the model learns from data and learning rate which is how long it goes without error going down. Important note to self: The lower the learning rate the longer it will take to train a model
#Parameters are the various connections between layers in the NN
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)


In [20]:
# Training the model
# How many times the model goes through all layers
numberOfRuns = 100
# Create a loss variable to see how well the model learns, however useless with current data
lossTrack = []
# Loop amount of times set in numberOfRuns
for i in range(numberOfRuns):
    y_pred = model.forward(X_train)
    # Optional loss things I found in a tutorial, good for tracking though
    loss = criterion(y_pred, y_train)
    lossTrack.append(loss.detach().numpy())
    
    if i % 10 == 0:
        print(f'Epoch: {i} and loss: {loss}')
    
    optimizer.zero_grad()
    optimizer.step()
    

Epoch: 0 and loss: 71.9935073852539
Epoch: 10 and loss: 71.9935073852539
Epoch: 20 and loss: 71.9935073852539
Epoch: 30 and loss: 71.9935073852539
Epoch: 40 and loss: 71.9935073852539
Epoch: 50 and loss: 71.9935073852539
Epoch: 60 and loss: 71.9935073852539
Epoch: 70 and loss: 71.9935073852539
Epoch: 80 and loss: 71.9935073852539
Epoch: 90 and loss: 71.9935073852539
